<a href="https://colab.research.google.com/github/Chromis07/dataStudy/blob/main/python/hands_on_data/%EB%84%A4%EC%9D%B4%EB%B2%84_%EC%98%81%ED%99%94_TOP100_%ED%81%AC%EB%A1%A4%EB%A7%81_%26_%EC%98%81%ED%99%94_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 링크 수집을 위한 라이브러리 설치
!pip install selenium

     |████████████████████████████████| 911kB 5.1MB/s 


라이브러리 임포트

In [ ]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver as wd
from selenium.webdriver.common.keys import Keys
import time
import re
import pandas as pd

In [ ]:
# install chromium, its driver, and selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium
# set options to be headless, ..
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome('chromedriver',options=options)

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15

분석할 데이터 모으기(크롤링)

네이버 영화 TOP100 페이지 별 링크 수집

In [ ]:
#https://serieson.naver.com/movie/top100List.nhn?&rankingTypeCode=PC_R&page= 실시간
#https://serieson.naver.com/movie/top100List.nhn?&rankingTypeCode=PC_D&page= 일간
#https://serieson.naver.com/movie/top100List.nhn?&rankingTypeCode=PC_W&page= 주간
#https://serieson.naver.com/movie/top100List.nhn?&rankingTypeCode=PC_M&page= 월간

def getPageLinksWantRange(startPageNo, lastPageNo) :
  links = [] # 100개의 영화 링크를 수집할 리스트 변수 선언
  
  for pageNo in range(startPageNo -1, lastPageNo) :
    # 실시간 TOP100 영화 링크 수집
    url = "https://serieson.naver.com/movie/top100List.nhn?&rankingTypeCode=PC_D&page=" + str(pageNo+1) #일간
    
    req = requests.get(url) # 페이지 접속
    soup = BeautifulSoup(req.text, 'lxml') # html 파싱을 위한 객체 생성
    movielinks = soup.select('div.lst_thum_wrap ul li a[href]') # 뒤 링크만 수집하는 단계. 앞 라인도 추가 필요
    # <div class="lst_thum_wrap"> -> <ul> -> <li> -> <a href=""> / <ul>와 <li>는 세트

    for movielink in movielinks :
      link = str(movielink.get('href'))
      links.append("https://serieson.naver.com/" + link) # 접속 가능한 전체 링크 형태로 변환하여 append
  
  return links

네이버 영화 제목, 평점, 장르, 줄거리 크롤링

In [ ]:
def getMovieDataFromNaverSeries(links) :
  title_infos = [] # 제목
  content_infos = [] # 줄거리
  genre_infos = [] # 장르
  score_infos = [] # 평점
  date_infos = [] # 개봉일

  url2 = "https://www.naver.com"

  driver = webdriver.Chrome('chromedriver', options=options)
  driver.get(url2)
  time.sleep(3.0) # 3초 sleep

  driver.find_element_by_css_selector('body').send_keys(Keys.CONTROL + 't') # 새 탭 생성

  for link in links :
    print(link + ' 수집 중......')
    driver.switch_to.window(driver.window_handles[-1]) # 새탭 주소창 활성화
    time.sleep(0.1)
    driver.get(link)
    time.sleep(0.1)
    driver.switch_to.window(driver.window_handles[0]) # 새탭 주소창 활성화
    time.sleep(3.0)

    html_source = driver.page_source # req.text 같은 동작
    html_soup = BeautifulSoup(html_source, 'lxml')

    flag = html_soup.text[0:10]

    # 청소년관림불가 영화 크롤링의 경우 -> 인증 -> 수집 제외

    newflag = ''.join(flag)
    newflag = newflag.replace('\n', '')
    
    if newflag == '네이버' :
      time.sleep(1.0)

      # 평점 수집
      score = driver.find_element_by_css_selector('div.score_area > em')
      score = float(score.text)
      score = int(score) # 소수점 원하면 주석 처리
      score_infos.append(score)
      print(score)

      # 장르 수집
      #soup은 > 꺾쇠 안써도 되지만, webdriver에서 사용
      genre = driver.find_element_by_css_selector('li.info_lst > ul > li:nth-child(4)').get_attribute('textContent')
      genre = genre.replace('장르', '') # '장르' 단어도 들어와버렸으니 전처리
      genre = genre.replace('\n', '')
      genre = genre.replace('\t', '')
      genre_infos.append(genre)

      # 제목, 줄거리 수집 (상세정보 탭으로 들어가 접근)
      try : 
        movieInfoUrl = driver.find_element_by_css_selector('li.info_lst > ul > li:nth-child(7) > a').get_attribute('href') # 7번째 li에 있는 링크 가져오기
      except :
        movieInfoUrl = driver.find_element_by_css_selector('li.info_lst > ul > li:nth-child(6) > a').get_attribute('href') # 없으면 그냥 6번째로 간다
      
      # 영화 상세보기 페이지로 이동
      movie_req = requests.get(movieInfoUrl)

      # 제목 수집
      movie_soup = BeautifulSoup(movie_req.text, 'lxml')
      title = movie_soup.head.find("meta",{"property":"og:title"}).get('content')
      title_infos.append(title)

      # 줄거리 수집
      contents_texts = movie_soup.select('div.story_area > p.con_tx') # f12 페이지 소스에서 확인!

      if len(contents_texts) == 0 :
        content_infos.append("줄거리 오류")
      
      else : 
        for contents in contents_texts :
          # 줄거리 데이터 클랜징 작업 (띄어쓰기 등)
          temp = contents.text
          temp = temp.replace("\r", '') # 줄바꿈 제거
          temp = temp.replace("\xa0", '') # 공백 제거
          content_infos.append(temp)
    
    elif newflag == '' :
      print('청불 영화로 데이터를 수집하지 않습니다')

  print('수집 완료......')
  print(len(score_infos), len(genre_infos), len(content_infos))
  driver.close()

  movie_dic = {"제목" : title_infos, "평점" : score_infos, "장르" : genre_infos, "줄거리" : content_infos}

  # 딕셔너리 -> DataFrames
  movie_df = pd.DataFrame(movie_dic)

  # 수집된 정보 중에 중복 데이터 삭제
  movie_df2 = movie_df.drop_duplicates("줄거리", keep='first') # 줄거리 비교하여, 중복된 영화를 삭제 (첫번째 수집 영화만 남김)

  return movie_df2

CSV 파일로 저장

In [ ]:
def dftoCsv(movie_df, num) :
  try :
    movie_df.to_csv('movie_my_data_'+str(num)+'.csv', 
                    sep=',', na_rep='NaN', encoding='euc-kr')
    
  except :
    print('Error')

크롤링 실행

In [ ]:
# 1) 크롤링 할 링크 수집

links = getPageLinksWantRange(1,5)

# 2) 영화 상세 정보 수집
movie_df2 = getMovieDataFromNaverSeries(links)

# 3) csv 파일 저장
dftoCsv(movie_df2, len(movie_df2))

# 시각화 (실시간)

In [3]:
# 라이브러리 임포트
import plotly.express as px
import plotly.express as go
import plotly.graph_objects as go
import numpy as np
import pandas as pd

In [18]:
# 실시간 TOP 100 영화데이터 불러오기
df1 = pd.read_csv('/content/movie_my_data_75_R.csv', encoding='euc-kr')
df1

,Unnamed: 0,제목,평점,장르,줄거리
0,0,콰이어트 플레이스,7,공포/스릴러,소리를 내는 순간 공격받는 극한의 상황 속에서 살아남기 위한 한 가족의 숨막히는 사...
1,1,극장판 귀멸의 칼날: 무한열차편,9,애니메이션,혈귀로 변해버린 여동생 ‘네즈코’를 인간으로 되돌릴 단서를 찾아 비밀조직 귀살대에 ...
2,2,킬러의 보디가드,8,액션,세상 제일 잘난 맛에 사는 섭외 1순위 앵그리 보디가드가지명수배 1순위 구강 액션 ...
3,3,크루엘라,9,드라마,처음부터 난 알았어. 내가 특별하단 걸그게 불편한 인간들도 있겠지만 모두의 비위를 ...
4,4,파이프라인,6,액션,손만 대면 대박을 터트리는 도유 업계 최고 천공기술자 ‘핀돌이’는수천억의 기름을 빼...
...,...,...,...,...,...
70,79,시간을 달리는 소녀,9,드라마,저 마코토에게는 남들에게 말 못할 비밀이 하나 있어요. 타임리프라고 하는 능력이죠....
71,80,쉬즈 더 맨,8,코미디,말괄량이 바이올라(아만다 바인즈)는 헤어진 남자친구에게 복수하기 위하여 쌍둥이 오빠...
72,81,해리 포터와 불의 잔,8,액션,해리 포터 일생일대 최대 난관!요즘 들어 매일 꾸는 악몽 때문에 이마의 상처에 더욱...
73,82,이터널 선샤인,9,드라마,조엘은 아픈 기억만을 지워준다는 라쿠나사를 찾아가 헤어진 연인 클레멘타인의 기억을 ...


In [21]:
# 평점 9점 데이터
df1_score = df1[df1.평점 == 9]
df1_score.tail()

,Unnamed: 0,제목,평점,장르,줄거리
60,69,세상에서 가장 아름다운 이별,9,드라마,"“피곤해” 병원 일에만 신경 쓰는 가장(김갑수)“밥 줘, 밥” 어린애가 되어버린 할..."
67,76,말할 수 없는 비밀,9,드라마,나는 너를 사랑해!너도 나를 사랑하니?예술학교로 전학 온 상륜(주걸륜)은 아버지의 ...
68,77,라따뚜이,9,액션,"절대미각, 빠른 손놀림, 끓어 넘치는 열정의 소유자 ‘레미’. 프랑스 최고의 요리사..."
70,79,시간을 달리는 소녀,9,드라마,저 마코토에게는 남들에게 말 못할 비밀이 하나 있어요. 타임리프라고 하는 능력이죠....
73,82,이터널 선샤인,9,드라마,조엘은 아픈 기억만을 지워준다는 라쿠나사를 찾아가 헤어진 연인 클레멘타인의 기억을 ...


In [23]:
# 영화 별 평점 시각화
fig = px.scatter(df1, x='제목', y='평점', color='장르')
fig.show()

In [22]:
# 실시간 TOP100 영화 별 평점 시각화
fig = px.bar(df1, x='제목', y='평점', color='장르')
fig.show()

In [24]:
fig = px.pie(df1, values='평점', names='장르')
fig.show()

# 시각화 (일간)

In [25]:
# 라이브러리 임포트
import plotly.express as px
import plotly.express as go
import plotly.graph_objects as go
import numpy as np
import pandas as pd

In [35]:
# 일간 TOP 100 영화데이터 불러오기
df2 = pd.read_csv('/content/movie_my_data_61_D.csv', encoding='euc-kr')
df2

,Unnamed: 0,제목,평점,장르,줄거리
0,0,극장판 귀멸의 칼날: 무한열차편,9,애니메이션,혈귀로 변해버린 여동생 ‘네즈코’를 인간으로 되돌릴 단서를 찾아 비밀조직 귀살대에 ...
1,1,콰이어트 플레이스,7,공포/스릴러,소리를 내는 순간 공격받는 극한의 상황 속에서 살아남기 위한 한 가족의 숨막히는 사...
2,2,킬러의 보디가드,8,액션,세상 제일 잘난 맛에 사는 섭외 1순위 앵그리 보디가드가지명수배 1순위 구강 액션 ...
3,3,크루엘라,9,드라마,처음부터 난 알았어. 내가 특별하단 걸그게 불편한 인간들도 있겠지만 모두의 비위를 ...
4,4,소울,9,애니메이션,뉴욕에서 음악 선생님으로 일하던 ‘조’는꿈에 그리던 최고의 밴드와 재즈 클럽에서 연...
...,...,...,...,...,...
56,70,비긴 어게인,9,코미디,싱어송라이터인 ‘그레타’(키이라 나이틀리)는 남자친구 ‘데이브’(애덤 리바인)가 메...
57,71,내가 널 사랑할 수 없는 10가지 이유,9,코미디,상냥하고 얼굴도 예쁜 비앙카(Bianca Stratford: 라리사 오레이닉 분)는...
58,72,내가 죽기를 바라는 자들,7,액션,화재 현장에서 세 명의 아이를 구하지 못한 죄책감과 트라우마로 감시탑에 배정된공수소...
59,73,애프터,7,드라마,첫 끌림대학 입학 후 첫 파티에 초대 받게 된 신입생 ‘테사’는진실게임을 하던 중 ...


In [27]:
# 평점 9점 데이터
df2_score = df2[df2.평점 == 9]
df2_score.tail()

,Unnamed: 0,제목,평점,장르,줄거리
51,63,밥정,9,다큐멘터리,"잔디, 잡초, 이끼, 나뭇가지.. 자연을 재료 삼아 요리를 만드는 방랑식객 임지호 ..."
54,66,스타 이즈 본,9,드라마,노래에 놀라운 재능을 가졌지만 외모에는 자신이 없는 무명가수 앨리(레이디 가가)는공...
55,68,원더,9,드라마,누구보다 위트 있고 호기심 많은 매력 부자 ‘어기'(제이콥 트렘블레이).하지만 남들...
56,70,비긴 어게인,9,코미디,싱어송라이터인 ‘그레타’(키이라 나이틀리)는 남자친구 ‘데이브’(애덤 리바인)가 메...
57,71,내가 널 사랑할 수 없는 10가지 이유,9,코미디,상냥하고 얼굴도 예쁜 비앙카(Bianca Stratford: 라리사 오레이닉 분)는...


In [30]:
# 영화 별 평점 시각화
fig = px.scatter(df2, x='제목', y='평점', color='장르')
fig.show()

In [29]:
# 영화 별 평점 시각화
fig = px.bar(df2, x='제목', y='평점', color='장르')
fig.show()

In [31]:
fig = px.pie(df2, values='평점', names='장르')
fig.show()

# 시각화 (주간)

In [32]:
# 라이브러리 임포트
import plotly.express as px
import plotly.express as go
import plotly.graph_objects as go
import numpy as np
import pandas as pd

In [37]:
# 주간 TOP 100 영화데이터 불러오기
df3 = pd.read_csv('/content/movie_my_data_63_W.csv', encoding='euc-kr')
df3

,Unnamed: 0,제목,평점,장르,줄거리
0,0,극장판 귀멸의 칼날: 무한열차편,9,애니메이션,혈귀로 변해버린 여동생 ‘네즈코’를 인간으로 되돌릴 단서를 찾아 비밀조직 귀살대에 ...
1,1,콰이어트 플레이스,7,공포/스릴러,소리를 내는 순간 공격받는 극한의 상황 속에서 살아남기 위한 한 가족의 숨막히는 사...
2,2,킬러의 보디가드,8,액션,세상 제일 잘난 맛에 사는 섭외 1순위 앵그리 보디가드가지명수배 1순위 구강 액션 ...
3,3,파이프라인,6,액션,손만 대면 대박을 터트리는 도유 업계 최고 천공기술자 ‘핀돌이’는수천억의 기름을 빼...
4,4,소울,9,애니메이션,뉴욕에서 음악 선생님으로 일하던 ‘조’는꿈에 그리던 최고의 밴드와 재즈 클럽에서 연...
...,...,...,...,...,...
58,73,태극기 휘날리며,9,액션,가족의 생계를 책임지기 위해 열심히 살아가는 ‘진태’(장동건)는 약혼녀 ‘영신’(이...
59,74,소년시절의 너,9,액션,시험만 잘 치면 멋진 인생을 살 수 있다고 가르치는 세상에서기댈 곳 없이 세상에 내...
60,76,하울의 움직이는 성,9,애니메이션,"어느 날, 영문도 모른 채 마녀의 저주로 인해 할머니가 된 소녀 '소피'절망 속에서..."
61,77,101마리의 달마시안 개,9,액션,악녀 크루엘라 드빌의 하수인에 달마시안 강아지들이 납치되고 강아지들의 부모인 퐁고와...


In [40]:
# 평점 9점 데이터
df3_score = df3[df3.평점 == 9]
df3_score.tail()

,Unnamed: 0,제목,평점,장르,줄거리
58,73,태극기 휘날리며,9,액션,가족의 생계를 책임지기 위해 열심히 살아가는 ‘진태’(장동건)는 약혼녀 ‘영신’(이...
59,74,소년시절의 너,9,액션,시험만 잘 치면 멋진 인생을 살 수 있다고 가르치는 세상에서기댈 곳 없이 세상에 내...
60,76,하울의 움직이는 성,9,애니메이션,"어느 날, 영문도 모른 채 마녀의 저주로 인해 할머니가 된 소녀 '소피'절망 속에서..."
61,77,101마리의 달마시안 개,9,액션,악녀 크루엘라 드빌의 하수인에 달마시안 강아지들이 납치되고 강아지들의 부모인 퐁고와...
62,78,나의 히어로 아카데미아 더 무비: 두 명의 히어로,9,애니메이션,"모두가 ‘개성’이라는 초능력을 가진 세계, 여기에 ‘무개성’의 소년이 있었다!하지만..."


In [48]:
# 영화 별 평점 시각화
fig = px.scatter(df3, x='제목', y='평점', color='장르')
fig.show()

In [42]:
# 영화 별 평점 시각화
fig = px.bar(df3, x='제목', y='평점', color='장르')
fig.show()

In [43]:
fig = px.pie(df3, values='평점', names='장르')
fig.show()

# 시각화 (월간)

In [44]:
# 라이브러리 임포트
import plotly.express as px
import plotly.express as go
import plotly.graph_objects as go
import numpy as np
import pandas as pd

In [45]:
# 월간 TOP 100 영화데이터 불러오기
df4 = pd.read_csv('/content/movie_my_data_64_M.csv', encoding='euc-kr')
df4

,Unnamed: 0,제목,평점,장르,줄거리
0,0,미나리,7,드라마,"""미나리는 어디서든 잘 자라""낯선 미국, 아칸소로 떠나온 한국 가족.가족들에게 뭔가..."
1,1,노바디,8,액션,비범한 과거를 숨긴 채 남들과 다를 바 없는 평범한 일상을 사는 한 가정의 가장 ‘...
2,2,자산어보,9,드라마,"“이 양반은 대역 죄인이니 너무 잘해줄 생각들 말어”순조 1년, 신유박해로 세상의 ..."
3,3,라야와 마지막 드래곤,9,애니메이션,"인간과 드래곤이 평화롭게 공존하던 신비의 땅, 쿠만드라 왕국살아있는 모든 생명을 삼..."
4,4,소울,9,애니메이션,뉴욕에서 음악 선생님으로 일하던 ‘조’는꿈에 그리던 최고의 밴드와 재즈 클럽에서 연...
...,...,...,...,...,...
59,71,위대한 쇼맨,9,드라마,"쇼 비즈니스의 창시자이자, 꿈의 무대로 전세계를 매료시킨 남자‘바넘’의 이야기에서 ..."
60,73,어바웃 타임,9,코미디,"모태솔로 팀(돔놀 글리슨)은 성인이 된 날, 아버지(빌 나이)로부터 놀랄만한 가문의..."
61,74,어벤져스,8,액션,"지구의 안보가 위협당하는 위기의 상황에서 슈퍼히어로들을 불러모아 세상을 구하는, 일..."
62,75,해리 포터와 비밀의 방,8,액션,해리 포터에겐 이번 여름방학이 별로 즐겁질 못했다. 마법이라면 질색을 하는 페투니아...


In [46]:
# 평점 9점 데이터
df4_score = df4[df4.평점 == 9]
df4_score.tail()

,Unnamed: 0,제목,평점,장르,줄거리
54,66,소년시절의 너,9,액션,시험만 잘 치면 멋진 인생을 살 수 있다고 가르치는 세상에서기댈 곳 없이 세상에 내...
55,67,뷰티풀 마인드,9,드라마,40년대 최고의 엘리트들이 모이는 프린스턴 대학원. 시험도 보지 않고 장학생으로 입...
56,68,원더,9,드라마,누구보다 위트 있고 호기심 많은 매력 부자 ‘어기'(제이콥 트렘블레이).하지만 남들...
59,71,위대한 쇼맨,9,드라마,"쇼 비즈니스의 창시자이자, 꿈의 무대로 전세계를 매료시킨 남자‘바넘’의 이야기에서 ..."
60,73,어바웃 타임,9,코미디,"모태솔로 팀(돔놀 글리슨)은 성인이 된 날, 아버지(빌 나이)로부터 놀랄만한 가문의..."


In [49]:
# 영화 별 평점 시각화
fig = px.scatter(df3, x='제목', y='평점', color='장르')
fig.show()

In [47]:
# 영화 별 평점 시각화
fig = px.bar(df4, x='제목', y='평점', color='장르')
fig.show()

In [50]:
fig = px.pie(df4, values='평점', names='장르')
fig.show()